In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

### 0. 크롬 드라이버 실행

In [2]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

### 1. 제품 링크 수집 (립메이크업 카테고리 페이지)

In [3]:
# 드라이버 실행
url = "https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100020006"
driver.get(url)
time.sleep(3)

# 상품 링크 수집 (20개 기준)
product_elements = driver.find_elements(By.CSS_SELECTOR, "ul.prd_list li .prd_name a")
product_links = [elem.get_attribute('href') for elem in product_elements]

print("📦 수집된 상품 수:", len(product_links))

# 데이터 저장용 리스트 
data = []

📦 수집된 상품 수: 0


### 2. 각 제품 상세 페이지에서 리뷰 탭 클릭 및 크롤링

In [4]:
for idx, link in enumerate(product_links):
    print(f"\n======================")
    print(f"[{idx+1}/{len(product_links)}] 상품 리뷰 수집 중: {link}")
    driver.get(link)
    time.sleep(2)

    try:
        # 리뷰 탭 클릭
        review_tab = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.goods_reputation'))
        )
        driver.execute_script("arguments[0].click();", review_tab)

        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'ul#gdasList li'))
        )
        print("✅ 리뷰 탭 로딩 완료")
    except:
        print("❌ 리뷰 탭 로딩 실패")
        continue

    # 리뷰 페이지 반복
    for page in range(5):  # 페이지당 최대 5페이지 (필요시 조절)
        time.sleep(2)
        review_items = driver.find_elements(By.CSS_SELECTOR, 'ul#gdasList li')
        print(f" - [Page {page+1}] 리뷰 {len(review_items)}개 수집")

        for review in review_items:
            try:
                # ID / 피부타입
                info_text = review.find_element(By.CSS_SELECTOR, '.info_user').text.strip()
                info_parts = info_text.split('\n')
                user_id = info_parts[0].strip()
                skin_type = info_parts[1].strip() if len(info_parts) > 1 else ""

                # 별점
                try:
                    star_style = review.find_element(By.CSS_SELECTOR, '.score .star span').get_attribute('style')
                    rating = int(int(star_style.split(':')[1].replace('%;', '').strip()) / 20)
                except:
                    rating = 0

                # 날짜
                date = review.find_element(By.CSS_SELECTOR, '.date').text.strip()

                # 평가 요소 (발색력, 지속력 등)
                tag_elements = review.find_elements(By.CSS_SELECTOR, '.tag_type .tag')
                tags = ", ".join([t.text.strip() for t in tag_elements])

                # 리뷰 텍스트 (.review_txt는 없고 .txt_inner가 있음)
                review_text = review.find_element(By.CSS_SELECTOR, '.txt_inner').text.strip()

                # 저장
                data.append({
                    'Product_URL': link,
                    'UserID': user_id,
                    'SkinType': skin_type,
                    'Rating': rating,
                    'Date': date,
                    'Tags': tags,
                    'Review': review_text
                })

            except Exception as e:
                print("⚠️ 리뷰 수집 오류:", e)
                continue

        # 다음 페이지 클릭
        try:
            next_btn = driver.find_element(By.CSS_SELECTOR, 'a.next')
            if 'disabled' in next_btn.get_attribute('class'):
                print("⛔ 다음 페이지 없음")
                break
            else:
                driver.execute_script("arguments[0].click();", next_btn)
        except:
            print("⛔ 다음 페이지 이동 실패")
            break

### 3. 결과 저장

In [5]:
driver.quit()

df = pd.DataFrame(data)
df.to_csv("oliveyoung_lip_reviews.csv", index=False, encoding='utf-8-sig')
print("\n✅ 전체 리뷰 크롤링 완료. 총 리뷰 수:", len(df))


✅ 전체 리뷰 크롤링 완료. 총 리뷰 수: 0
